<a href="https://colab.research.google.com/github/wendirad/weekTwo/blob/task_2/notebooks/task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import  pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [6]:
da = pd.read_csv('da.csv')

In [7]:
session_frequency = da.groupby('Bearer Id').nunique()
session_duration = da.groupby('Bearer Id')['Dur. (ms)'].sum()